In [56]:
# instalar dependencias
!pip install pandas
!pip install jax
!pip install --upgrade "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install scikit-learn


/usr/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


In [57]:
import pandas as pd
import jax.numpy as jnp
import numpy as np

# Carregando o dataset Iris diretamente do scikit-learn
from sklearn.datasets import load_wine
wine = load_wine()

# Convertendo para um DataFrame do Pandas para facilitar a manipulação
data = pd.DataFrame(data= np.c_[wine['data'], wine['target']],
                    columns= wine['feature_names'] + ['target'])

# Convertendo para um array JAX se necessário para cálculos futuros
data_jax = jnp.array(data.values)

#imprimir o shape de data
print(data.shape)

# Exibindo as primeiras linhas do DataFrame
print(data.head())

(178, 14)
   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   od280/od315_of_diluted_wines  proline  target  
0

In [87]:
#separa treino e teste
from sklearn.model_selection import train_test_split

X = data.drop('target', axis=1) # features
y = data['target'] # target

X_train_original, X_test_original, y_train_original, y_test_original = train_test_split(X, y, test_size=0.2, random_state=42)

# transforma em array
print(X_train_original)
X_train = jnp.array(X_train_original)
print(X_train[:, 17]) # imprime a primeira coluna do conjunto de treino
X_test = jnp.array(X_test_original)
y_train = jnp.array(y_train_original)
y_test = jnp.array(y_test_original)

     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
158    14.34        1.68  2.70               25.0       98.0           2.80   
137    12.53        5.51  2.64               25.0       96.0           1.79   
98     12.37        1.07  2.10               18.5       88.0           3.52   
159    13.48        1.67  2.64               22.5       89.0           2.60   
38     13.07        1.50  2.10               15.5       98.0           2.40   
..       ...         ...   ...                ...        ...            ...   
71     13.86        1.51  2.67               25.0       86.0           2.95   
106    12.25        1.73  2.12               19.0       80.0           1.65   
14     14.38        1.87  2.38               12.0      102.0           3.30   
92     12.69        1.53  2.26               20.7       80.0           1.38   
102    12.34        2.45  2.46               21.0       98.0           2.56   

     flavanoids  nonflavanoid_phenols  proanthocyan

In [59]:
def native_knn(X_train, X_test, y_train, k):
    from sklearn.neighbors import KNeighborsClassifier

    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    return y_pred

def evaluate(y_true, y_pred):
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_true, y_pred)
    

def generalized_knn(X_train, y_train, X_test, k, metric):
   # metric is a function that takes two arguments and returns a scalar
    print(X_train.dtype)
    distances = jnp.array([[metric(x, x_train) for x_train in X_train] for x in X_test])
    print(distances)
    return distances


# print(knn(X_train, X_test, y_train, y_test, 3))

manhattan = lambda x, y: jnp.sum(jnp.abs(x - y))
euclidean = lambda x, y: jnp.sqrt(jnp.sum((x - y) ** 2))


print(generalized_knn(X_train, y_train, X_test, 3, euclidean))

float32
[[ 186.32301   330.7766    187.15723  ...  702.15436   351.90906
   407.4495  ]
 [ 420.3308    565.2437    420.64777  ...  467.15283   585.83966
   642.14465 ]
 [  60.8279     85.142265   60.74745  ...  947.07764   106.24117
   162.03983 ]
 ...
 [ 855.083    1000.08234   855.1946   ...   33.05907  1020.3432
  1077.0411  ]
 [ 660.1677    805.09546   660.0097   ...  227.42169   825.06964
   882.0665  ]
 [ 435.2018    580.13367   435.2139   ...  452.01837   600.4092
   657.0437  ]]
[[ 186.32301   330.7766    187.15723  ...  702.15436   351.90906
   407.4495  ]
 [ 420.3308    565.2437    420.64777  ...  467.15283   585.83966
   642.14465 ]
 [  60.8279     85.142265   60.74745  ...  947.07764   106.24117
   162.03983 ]
 ...
 [ 855.083    1000.08234   855.1946   ...   33.05907  1020.3432
  1077.0411  ]
 [ 660.1677    805.09546   660.0097   ...  227.42169   825.06964
   882.0665  ]
 [ 435.2018    580.13367   435.2139   ...  452.01837   600.4092
   657.0437  ]]
